Messing around with some functionality to incorporate into the webapp

I have a suspicion that scikitlearn will be more performant for actually serving predictions. StatsModels was great for understanding but scikit is more built for production. Let's find out

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import d6tflow

Loading postgres module without psycopg2 installed. Will crash at runtime if postgres functionality is used.
Loading S3 module without the python package boto3. Will crash at runtime if S3 functionality is used.


Welcome to d6tflow!


In [3]:
from terra_mystica_models.models.train_model import TaskSkLearnModel, TaskScoreTurnModel
from terra_mystica_models.models.analysis import TaskScoreTurnModelInputs

In [4]:
# Run these once to generate pickle files
d6tflow.run(TaskSkLearnModel())
d6tflow.run(TaskScoreTurnModelInputs())

INFO: Informed scheduler that task   TaskSkLearnModel__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
INFO: Worker Worker(salt=078014271, workers=1, host=terra, username=Ian, pid=6872) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 complete ones were encountered:
    - 1 TaskSkLearnModel()

Did not run any tasks
This progress looks :) because there were no failed tasks or missing dependencies

===== Luigi Execution Summary =====

INFO: Informed scheduler that task   TaskScoreTurnModelInputs__99914b932b   has status   DONE
INFO: Done scheduling tasks
INFO: Running Worker with 1 processes
INFO: Worker Worker(salt=414272247, workers=1, host=terra, username=Ian, pid=6872) was stopped. Shutting down Keep-Alive thread
INFO: 
===== Luigi Execution Summary =====

Scheduled 1 tasks of which:
* 1 complete ones were encountered:
    - 1 TaskScoreTurnModelInputs()

Did not ru

True

The idea is that I want to generate pickled artifacts of anything I'll need from the trained models. That way I can just copy them into a container or whatever and deploy something without all these training/exploration requirements

Right now the way I'm loading it up is a little hacky. I'll have to do something to export the file into the webapp folder with clean names

In [5]:
from terra_mystica_models import practice_app as pa

In [6]:
sk2 = pa.load_model_skl()

In [7]:
sm2 = pa.load_model_sm()

In [8]:
test_input = pa.generate_input_series("SCORE2", "SCORE3", "SCORE4", "SCORE5", "SCORE6", "SCORE7", ["BON1", "BON2", "BON3", "BON4", "BON5", "BON6", "BON7"], "darklings")

In [9]:
sk2.predict(test_input)[0]

6.905517578125

In [10]:
sm2.predict(test_input)[0]

6.948226348048324

In [11]:
%timeit sk2.predict(test_input)[0]

11.1 ms ± 227 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [12]:
%timeit sm2.predict(test_input)[0]

166 µs ± 3.34 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


Oh wow, I'm glad I checked. Statsmodels is way faster! That's confusing but nice.

The pickled sklearn model is way smaller, a few KB vs a few MB, but I'd rather have a bigger image that can generate faster predictions. That's super weird, I'm really surprised statsmodels is so much faster.

While I'm doing this sort of testing, let's see how long it's taking me to generate the input series

In [13]:
%timeit pa.generate_input_series("SCORE2", "SCORE3", "SCORE4", "SCORE5", "SCORE6", "SCORE7", ["BON1", "BON2", "BON3", "BON4", "BON5", "BON6", "BON7"], "darklings")

2.42 ms ± 156 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Ok, if I was using scikit that would be the bottleneck, but using statsmodels this is actually the slower part. That's interesting.